# 0. Configuration

In [2]:
# links to shared data MovieLens
# source on kaggle: https://www.kaggle.com/code/quangnhatbui/movie-recommender/data
MOVIES_METADATA_URL = 'https://drive.google.com/file/d/19g6-apYbZb5D-wRj4L7aYKhxS-fDM4Fb/view?usp=share_link'
RATINGS_SMALL_URL = 'https://drive.google.com/file/d/1BlZfCLLs5A13tbNSJZ1GPkHLWQOnPlE4/view?usp=share_link'

# 1. Modules and functions

In [3]:
import re
import nltk
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from ast import literal_eval
from pymystem3 import Mystem
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import warnings
warnings.filterwarnings('ignore')

# download stop words beforehand
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nikitasenyatkin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 1.1. Helper functions to avoid copypaste

In [4]:
def read_csv_from_gdrive(url):
    """
    gets csv data from a given url (taken from file -> share -> copy link)
    :url: example https://drive.google.com/file/d/1BlZfCLLs5A13tbNSJZ1GPkHLWQOnPlE4/view?usp=share_link
    """
    file_id = url.split('/')[-2]
    file_path = 'https://drive.google.com/uc?export=download&id=' + file_id
    data = pd.read_csv(file_path)

    return data

In [5]:
# init lemmatizer to avoid slow performance
mystem = Mystem() 

def word_tokenize_clean(doc: str, stop_words: list):
    '''
    tokenize from string to list of words
    '''

    # split into lower case word tokens \w lemmatization
    tokens = list(set(mystem.lemmatize(doc.lower())))
  
    # remove tokens that are not alphabetic (including punctuation) and not a stop word
    tokens = [word for word in tokens if word.isalpha() and not word in stop_words \
              not in list(punctuation)]
    return tokens

# 2. Main

## 2.1. Data Preparation

In [6]:
# read csv information about films etc
movies_metadata = read_csv_from_gdrive(MOVIES_METADATA_URL)
movies_metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [7]:
# let's see what columns we have
movies_metadata.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

To get accurate results we need to preprocess text a bit. The pipeline will be as follows:

- Filter only necessary columns from movies_metadada : id, original_title, overview;
- Define `model_index` for model to match back with `id` column;
- Text cleaning: removing stopwords & punctuation, lemmatization for further tokenization and tagged document creatin required for gensim.Doc2Vec

In [8]:
# filter cols
sample = movies_metadata[['id', 'original_title', 'overview']].copy()
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              45466 non-null  object
 1   original_title  45466 non-null  object
 2   overview        44512 non-null  object
dtypes: object(3)
memory usage: 1.0+ MB


In [9]:
# as you see from above, we have missing overview in some cases -- let's fill it with the original title
sample.loc[sample['overview'].isnull(), 'overview'] = sample.loc[sample['overview'].isnull(), 'original_title']
sample.isnull().sum()

id                0
original_title    0
overview          0
dtype: int64

In [10]:
# define model_index and make it as string
sample = sample.reset_index().rename(columns = {'index': 'model_index'})
sample['model_index'] = sample['model_index'].astype(str)

In [11]:
# create mapper with title and model_idnex to use it further in evaluation
movies_inv_mapper = dict(zip(sample['original_title'].str.lower(), sample['model_index'].astype(int)))

In [12]:
# preprocess by removing non-character data, stopwords
tags_corpus = sample['overview'].values
tags_corpus = [re.sub('-[!/()0-9]', '', x) for x in tags_corpus]
stop_words = stopwords.words('english')

tags_doc = [word_tokenize_clean(description, stop_words) for description in tags_corpus]
tags_doc[:5]

[['buzz',
  'onto',
  'happily',
  'room',
  'live',
  'place',
  'losing',
  'circumstances',
  'woody',
  'duo',
  'heart',
  'plots',
  'andy',
  'lightyear',
  'learns',
  'owner',
  'put',
  'differences',
  'eventually',
  'scene',
  'aside',
  'toys',
  'birthday',
  'separate',
  'afraid',
  'led',
  'brings'],
 ['rhinoceroses',
  'monkeys',
  'creatures',
  'giant',
  'finish',
  'room',
  'game',
  'invite',
  'enchanted',
  'evil',
  'world',
  'terrifying',
  'risky',
  'judy',
  'discover',
  'board',
  'running',
  'hope',
  'proves',
  'unwittingly',
  'peter',
  'alan',
  'find',
  'siblings',
  'magical',
  'three',
  'opens',
  'trapped',
  'door',
  'inside',
  'adult',
  'freedom',
  'living',
  'years'],
 ['buddies',
  'bait',
  'shop',
  'interested',
  'wedding',
  'feud',
  'meanwhile',
  'scare',
  'ancient',
  'italian',
  'next',
  'sultry',
  'locals',
  'neighbors',
  'john',
  'seafood',
  'less',
  'reignites',
  'local',
  'hot',
  'family',
  'divorcée'

In [13]:
# prepare data as model input for Word2Vec
## it takes some time to execute
tags_doc = [TaggedDocument(words = word_tokenize_clean(D, stop_words), tags = [str(i)]) for i, D in enumerate(tags_corpus)]

In [14]:
# let's check what do we have
## tag = movie index
tags_doc[1]

TaggedDocument(words=['rhinoceroses', 'monkeys', 'creatures', 'giant', 'finish', 'room', 'game', 'invite', 'enchanted', 'evil', 'world', 'terrifying', 'risky', 'judy', 'discover', 'board', 'running', 'hope', 'proves', 'unwittingly', 'peter', 'alan', 'find', 'siblings', 'magical', 'three', 'opens', 'trapped', 'door', 'inside', 'adult', 'freedom', 'living', 'years'], tags=['1'])

# 2.2. Model Training and Evaluation

In [15]:
VEC_SIZE = 50
ALPHA = .02
MIN_ALPHA = .00025
MIN_COUNT = 5
EPOCHS = 20

In [16]:
# initialize
model = Doc2Vec(vector_size = VEC_SIZE,
                alpha = ALPHA, 
                min_alpha = MIN_ALPHA,
                min_count = MIN_COUNT,
                dm = 0)

In [17]:
# generate vocab from all tag docs
model.build_vocab(tags_doc)

In [18]:
# train model
model.train(tags_doc,
            total_examples = model.corpus_count,
            epochs = EPOCHS)

## 2.3. Evaluate the model

Let's assume that we watched movie `batman` and based on that generate recommendation similar to it's description.

To do that we need
- To extract movie id from `movies_inv_mapper` we created to map back titles from model output
- Load embeddings from trained model
- Use built-in most_similar() method to get most relevant recommendations based on film embedding
- Finally, map title names for sense-check

In [19]:
# get id
movie_id = movies_inv_mapper['batman']
movie_id

8603

In [20]:
# load trained embeddings 
movies_vectors = model.dv.vectors

In [21]:
movie_embeddings = movies_vectors[movie_id]

In [22]:
movie_embeddings

array([-0.0810033 , -0.15662926,  0.22732492, -0.03314524, -0.08831248,
        0.04698127, -0.37611416,  0.03127491, -0.3240497 ,  0.16387098,
       -0.27739313,  0.14243348, -0.02955425, -0.17033695,  0.19981188,
       -0.13149084,  0.2568314 , -0.24197924,  0.03781102, -0.19469059,
        0.07794127,  0.09775072, -0.09510607,  0.1666584 ,  0.22830246,
        0.27411062, -0.10825751, -0.07948952,  0.22657228, -0.25904343,
        0.04363595, -0.01238974,  0.00905011,  0.06264653, -0.2956632 ,
        0.28322053, -0.01838639, -0.10268471,  0.20658082,  0.13180552,
        0.15559144,  0.08908886, -0.04824213,  0.11570384,  0.01670172,
       -0.23906709,  0.08832628, -0.0444885 , -0.05377237,  0.49770957],
      dtype=float32)

In [23]:
# get recommendations
similars = model.docvecs.most_similar(positive = [movie_embeddings], topn = 20)
output = pd.DataFrame(similars, columns = ['model_index', 'model_score'])
output.head()

,model_index,model_score
0,8603,1.000000
1,13835,0.963732
2,43165,0.960418
3,5713,0.959865
4,44339,0.959307


In [24]:
# reverse values and indices to map names in dataframe
name_mapper = {v: k for k, v in movies_inv_mapper.items()}

In [25]:
output['title_name'] = output['model_index'].astype(int).map(name_mapper)
output


,model_index,model_score,title_name
0,8603,1.000000,batman
1,13835,0.963732,k2
2,43165,0.960418,the zookeeper's wife
3,5713,0.959865,rollover
4,44339,0.959307,the underground world
5,8916,0.957473,killer klowns from outer space
6,7772,0.952718,this island earth
7,30846,0.952314,scooby-doo! and the cyber chase
8,11256,0.952277,sun faa sau si
9,19227,0.952157,carbon nation


# TODO

- Add `original_title`, `keywords`, `tagline` and other metadata to train sample and then retrain embeddings;
- Make visualization of embeddings with links of films with each other;
- Compare results with the embeddings we created in lecture
- Write function get_recommendations() which takes arguments we used 2.3., but such that we can use embeddings of several watched films to get recommendations

# Appendix

Here, we wrap up all pipeline into functions to re-use if needed and it is just prettier to code this way :)

## Making personal rekkos (building get_recommendations() function that will give personal rekkos for each user with filter on watched films)

In [26]:
def get_clean_tags_array(agg_tags: pd.DataFrame,
                         text_col = 'overview'):
    '''text preprocessing
    '''
    tags_corpus = agg_tags[text_col].values
    tags_corpus = [re.sub('-[!/()0-9]', '', x) for x in tags_corpus]
    stop_words = stopwords.words('english')


    # preprocess corpus of movie tags before feeding it into Doc2Vec model
    tags_doc = [TaggedDocument(words = word_tokenize_clean(D, stop_words), tags = [str(i)]) for i, D in enumerate(tags_corpus)]

    return tags_doc


In [27]:
def train_embeddings(tags_doc: np.array,
                     epochs = 20,
                     vec_size = 50,
                     alpha = .02,
                     min_alpha =  0.00025,
                     min_count = 5,
                     save_path: str = None):
    """
    fit doc2vec model to prepared corpus
    :tags_doc: result of get_clean_tags_array()
    :max_epocs: int
    :vec_size: int
    :alpha: float
    """
    #initialize
    model = Doc2Vec(vector_size = vec_size,
                    alpha = alpha, 
                    min_alpha = min_alpha,
                    min_count = min_count,
                    dm = 0)
    
    #generate vocab from all tag docs
    model.build_vocab(tags_doc)
    
    #train model
    model.train(tags_doc,
                total_examples = model.corpus_count,
                epochs = epochs)
    
    #save model to dir
    if save_path:
        model.save(f'{save_path}/d2v_model.pkl')
    
    return model

In [28]:
tags_doc = get_clean_tags_array(sample)


In [29]:
model = train_embeddings(tags_doc)

lets load interactions data

In [30]:
interactions = read_csv_from_gdrive(RATINGS_SMALL_URL)
interactions['movieId'] = interactions['movieId'].astype(str)
movies_metadata.rename(columns = {'id': 'movieId'}, inplace = True)
interactions_filtered = interactions.loc[interactions['movieId'].isin(sample['model_index'])]
print(interactions.shape, interactions_filtered.shape)

(100004, 4) (88299, 4)


In [31]:
# create users input
users = interactions_filtered[['userId']].drop_duplicates().reset_index(drop = True)

In [32]:
known_items = interactions_filtered.groupby('userId')['movieId'].apply(list).to_dict()

users['watched_movies'] = users['userId'].map(known_items)
users.head(5)

,userId,watched_movies
0,1,"[31, 1029, 1061, 1129, 1172, 1263, 1287, 1293,..."
1,2,"[10, 17, 39, 47, 50, 52, 62, 110, 144, 150, 15..."
2,3,"[60, 110, 247, 267, 296, 318, 355, 356, 377, 5..."
3,4,"[10, 34, 112, 141, 153, 173, 185, 260, 289, 29..."
4,5,"[3, 39, 104, 141, 150, 231, 277, 344, 356, 364..."


In [33]:
movie_vectors = model.dv.vectors

In [34]:
# Defining a function to get the embeddings of a list of movies
def get_movie_embeddings(movie_list):
    embeddings = [movie_vectors[int(movie_id)] for movie_id in movie_list]
    # Remove None values
    embeddings = [x for x in embeddings if x is not None]
    return embeddings


users['watched_movies_embeddings'] = users['watched_movies'].apply(get_movie_embeddings)


In [35]:
users.head(5)

,userId,watched_movies,watched_movies_embeddings
0,1,"[31, 1029, 1061, 1129, 1172, 1263, 1287, 1293,...","[[0.050337248, -0.18196894, 0.12943622, -0.076..."
1,2,"[10, 17, 39, 47, 50, 52, 62, 110, 144, 150, 15...","[[-0.0488073, -0.26256245, 0.29426113, -0.0007..."
2,3,"[60, 110, 247, 267, 296, 318, 355, 356, 377, 5...","[[-0.084063806, -0.12840259, 0.20841043, 0.005..."
3,4,"[10, 34, 112, 141, 153, 173, 185, 260, 289, 29...","[[-0.0488073, -0.26256245, 0.29426113, -0.0007..."
4,5,"[3, 39, 104, 141, 150, 231, 277, 344, 356, 364...","[[-0.07024017, -0.091821566, 0.32127783, 0.005..."


In [36]:
## lets calculate mean embeddings for each user:

mean_embeddings = []
for embedding_list in users['watched_movies_embeddings']:
    embeddings = np.array(embedding_list)
    # Take the mean of the embeddings for movies that the user has watched
    user_embedding = np.mean(embeddings, axis=0)
    mean_embeddings.append(user_embedding)
    
# Add the mean_embeddings column to dataframe
users['mean_embeddings'] = mean_embeddings

In [37]:
#mapping titles and ids for furher function
name_mapper = dict(zip(sample['model_index'], sample['original_title']))

## Function that creates recommendations for each user

In [38]:
def rekkos(data: pd.DataFrame, number_of_samples: int):
    similars = model.docvecs.most_similar(positive=data, topn = number_of_samples)
    return [name_mapper.get(movie_id[0], movie_id[0]) for movie_id in similars]

let's calculate maximum films that we should predict to recommend every user fresh films:

In [39]:
lst_max = []
for row in users.index:
    lst_max.append(len(users['watched_movies'][row]))

max_films = max(lst_max)+20
max_films


1888

In [40]:
def watched_films_filter(df: pd.DataFrame, number_of_samples: int):
    """
    calculates mean rating to define popular titles with taking to the account watched films
    """
    popular_titles = df['rekkos']
    personal_rekkos = {}
    for key in known_items.keys():
        popular_titles = df['rekkos'][key-1]
        list_of_unknown_films = [x for x in popular_titles if x not in known_items[key]]
        personal_rekkos[key] = list(list_of_unknown_films[:number_of_samples])
    return personal_rekkos


## FINAL FUNCTION 

In [41]:
def get_recommendatios(data: pd.DataFrame, 
                       number_of_samples: int,
                       item_column: str):
    
    '''function that returns dataframe with recommended films for each user based on their previous history of watches.
    :data : pd.DataFrame 
    :number_of_samples : number of films needed to be predicted,
    :item_column : column with watched films in df.
    '''

    # creating embeddings for each movie
    data['watched_movies_embeddings'] = data[item_column].apply(get_movie_embeddings)

    #counting mean embeddings for watched_films
    mean_embeddings = []
    for embedding_list in data['watched_movies_embeddings']:
        embeddings = np.array(embedding_list)
        # Take the mean of the embeddings for movies that the user has watched
        user_embedding = np.mean(embeddings, axis=0)
        mean_embeddings.append(user_embedding)
    
    # Adding the mean_embeddings column to dataframe
    data['mean_embeddings'] = mean_embeddings

    #receiving rekkos
    data['rekkos'] = data['mean_embeddings'].apply(rekkos, number_of_samples=max_films)

    #filtering watched films 
    personal_rekkos = watched_films_filter(data, number_of_samples=number_of_samples)
    data['rekkos'] = data['userId'].map(personal_rekkos)
    
    return data[['userId', 'rekkos']]

In [42]:
recommended_df = get_recommendatios(data= users,number_of_samples= 20, item_column='watched_movies')
recommended_df.head(5)

,userId,rekkos
0,1,"[Tikhaya Zastava, Questo piccolo grande amore,..."
1,2,"[The Tiger's Tail, Hawaiian Vacation, Bon Bini..."
2,3,"[The Tiger's Tail, Don't Hug Me I'm Scared 5, ..."
3,4,"[Arthur et la vengeance de Maltazard, Tempo in..."
4,5,"[Don't Hug Me I'm Scared 5, Se tutto va bene s..."


## Adding more features into the model (this model works worse that first one, that is why I used first in recommendations)

In [43]:
movies_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   movieId                45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [44]:
new_sample = movies_metadata[['movieId', 'original_title', 'overview', 'genres', 'tagline']]

In [45]:
new_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   movieId         45466 non-null  object
 1   original_title  45466 non-null  object
 2   overview        44512 non-null  object
 3   genres          45466 non-null  object
 4   tagline         20412 non-null  object
dtypes: object(5)
memory usage: 1.7+ MB


In [46]:
new_sample.loc[new_sample['overview'].isnull(), 'overview'] = new_sample.loc[new_sample['overview'].isnull(), 'original_title']
new_sample['tagline'] = new_sample['tagline'].fillna(new_sample['original_title'])

In [47]:
new_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   movieId         45466 non-null  object
 1   original_title  45466 non-null  object
 2   overview        45466 non-null  object
 3   genres          45466 non-null  object
 4   tagline         45466 non-null  object
dtypes: object(5)
memory usage: 1.7+ MB


In [48]:
new_sample = new_sample.reset_index().rename(columns = {'index': 'model_index'})
new_sample['model_index'] = new_sample['model_index'].astype(str)
new_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   model_index     45466 non-null  object
 1   movieId         45466 non-null  object
 2   original_title  45466 non-null  object
 3   overview        45466 non-null  object
 4   genres          45466 non-null  object
 5   tagline         45466 non-null  object
dtypes: object(6)
memory usage: 2.1+ MB


In [49]:
movies_inv_mapper = dict(zip(new_sample['original_title'].str.lower(), new_sample['model_index'].astype(int)))

In [50]:
def get_clean_tags_array(agg_tags: pd.DataFrame,
                         text_cols = ['overview', 'original_title', 'tagline']):
    '''text preprocessing
    '''
    # concatenate the text from all columns of interest
    tags_corpus = agg_tags[text_cols].apply(lambda x: ' '.join(x), axis=1).values
    
    # preprocess the text
    tags_corpus = [re.sub('-[!/()0-9]', '', x) for x in tags_corpus]
    stop_words = stopwords.words('english')
    tags_doc = [TaggedDocument(words = word_tokenize_clean(D, stop_words), tags = [str(i)]) for i, D in enumerate(tags_corpus)]

    return tags_doc

In [51]:
tags_array = get_clean_tags_array(new_sample)

In [52]:
def train_embeddings(tags_doc: np.array,
                     epochs = 20,
                     vec_size = 50,
                     alpha = .02,
                     min_alpha =  0.00025,
                     min_count = 5,
                     save_path: str = None):
    """
    fit doc2vec model to prepared corpus
    :tags_doc: result of get_clean_tags_array()
    :max_epocs: int
    :vec_size: int
    :alpha: float
    """
    #initialize
    model = Doc2Vec(vector_size = vec_size,
                    alpha = alpha, 
                    min_alpha = min_alpha,
                    min_count = min_count,
                    dm = 0)
    
    #generate vocab from all tag docs
    model.build_vocab(tags_doc)
    
    #train model
    model.train(tags_doc,
                total_examples = model.corpus_count,
                epochs = epochs)
    
    #save model to dir
    if save_path:
        model.save(f'{save_path}/d2v_model.pkl')
    
    return model

In [53]:
model = train_embeddings(tags_array)

In [54]:
movie_id = movies_inv_mapper['batman']
movie_id

8603

In [55]:
movie_vectors = model.dv.vectors

In [56]:
movie_embeddings = movies_vectors[movie_id]

In [57]:
movie_embeddings

array([-0.0810033 , -0.15662926,  0.22732492, -0.03314524, -0.08831248,
        0.04698127, -0.37611416,  0.03127491, -0.3240497 ,  0.16387098,
       -0.27739313,  0.14243348, -0.02955425, -0.17033695,  0.19981188,
       -0.13149084,  0.2568314 , -0.24197924,  0.03781102, -0.19469059,
        0.07794127,  0.09775072, -0.09510607,  0.1666584 ,  0.22830246,
        0.27411062, -0.10825751, -0.07948952,  0.22657228, -0.25904343,
        0.04363595, -0.01238974,  0.00905011,  0.06264653, -0.2956632 ,
        0.28322053, -0.01838639, -0.10268471,  0.20658082,  0.13180552,
        0.15559144,  0.08908886, -0.04824213,  0.11570384,  0.01670172,
       -0.23906709,  0.08832628, -0.0444885 , -0.05377237,  0.49770957],
      dtype=float32)

In [58]:
similars = model.docvecs.most_similar(positive=[movie_embeddings], topn=20)
output = pd.DataFrame(similars, columns = ['model_index', 'model_score'])
output.head()

,model_index,model_score
0,33282,0.928115
1,39367,0.927254
2,9851,0.924900
3,37524,0.924483
4,44962,0.924437


In [59]:
output['title_name'] = output['model_index'].astype(int).map(name_mapper)
output

,model_index,model_score,title_name
0,33282,0.928115,NaN
1,39367,0.927254,NaN
2,9851,0.924900,NaN
3,37524,0.924483,NaN
4,44962,0.924437,NaN
5,11256,0.923972,NaN
6,26189,0.923584,NaN
7,16369,0.922782,NaN
8,28418,0.920941,NaN
9,29395,0.920029,NaN


As we can see, model with more attributes works worse, it doesn't have even batman by itself

In [60]:
import umap

: 

: 